In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
plt.style.use('seaborn')
import datetime as dt
import string
import pandas as pd
import numpy as np
import pygsheets
from pandas.tseries.offsets import BDay
import datetime as dt
import PyWinDDE

import pygsheets
gc = pygsheets.authorize(service_file='dividend_gspread.json')

survey_url = 'https://docs.google.com/spreadsheets/d/1GksJ5z94MOzLNcIEct_x_aWoWzBi4CEiHJUHjU7l0IQ/edit#gid=0'
sh = gc.open_by_url(survey_url)

path='//192.168.60.83/Wellington/Allen/Index'
twn_path='//192.168.60.83/Wellington/Allen/Index/BloombergTWNI/'
index_list=os.listdir(r'\\192.168.60.83\Wellington\Allen\Index')
TWN_list=os.listdir(r'\\192.168.60.83\Wellington\Allen\Index\BloombergTWNI')

今日資料日期=((datetime.datetime.today())).date()
# 今日資料日期 = datetime.date(2023,3,30)
today_readmode=今日資料日期.strftime('%b %d %Y')

TX_use_data = sorted([i for i in index_list if ('NU' in i and 'C.XLS' in i)],reverse=True)[0]
TWN_use_data = [i for i in TWN_list if today_readmode in i][0]
MTW_use_data = sorted([i for i in index_list if 'tw_per' in i],reverse=True)[0]

print('TX_Use:{} \nTWN_Use:{} \nMTW_Use:{}'.format(TX_use_data,TWN_use_data,MTW_use_data))

MTW=pd.read_excel(path+'/'+MTW_use_data,header=0,index_col=0).dropna(thresh=3,axis=0).iloc[1][1]

加權指數成分股=pd.read_html(path+'/'+TX_use_data,encoding='cp950',header=0)[-1]
加權指數成分股.columns=加權指數成分股.iloc[0]
加權指數成分股=加權指數成分股.iloc[1:]
加權指數成分股['Sector Code']=加權指數成分股['Sector Code'].astype(int)
加權指數成分股=加權指數成分股.set_index('Local Code')

富台指數成分股=pd.read_excel(twn_path+'/'+TWN_use_data,header=0,index_col=0)[["股數","價格"]]
富台指數成分股.index=[i.split(" ")[0] for i in 富台指數成分股.index]
富台指數成分股.columns=['Shares',"Close"]

# 富台指數成分股=pd.read_csv(twn_path+'/'+TWN_use_data, header=0, skiprows=2)[["BBG Ticker","Index Shares","Price"]]
# print(富台指數成分股['BBG Ticker'])
# 富台指數成分股.index=[str(i).split(" ")[0] for i in 富台指數成分股['BBG Ticker']]
# print(富台指數成分股)
# 富台指數成分股['BBG Ticker']
# 富台指數成分股.columns=['Shares',"Close"]
industry = pd.read_excel("./個股產業別.xlsx")

TX總市值 = 加權指數成分股['Market Cap. (Unit: NT$Thousands)'].apply(lambda x : float(x)).sum()
加權指數成分股['weight_%'] = 加權指數成分股['Market Cap. (Unit: NT$Thousands)'].apply(lambda x : float(x))/TX總市值
富台指數成分股['weight_%'] = ((富台指數成分股.Shares)*(富台指數成分股.Close))/((富台指數成分股.Shares)*(富台指數成分股.Close)).sum()

加權指數成分股['Code'] = 加權指數成分股.index
富台指數成分股['Code'] = 富台指數成分股.index

industry['Code'] = industry['Code'].apply(lambda x :str(x))

加權指數成分股 = 加權指數成分股.reset_index(drop = True)
富台指數成分股 = 富台指數成分股.reset_index(drop = True)

加權指數成分股['Code'] = 加權指數成分股['Code'].apply(lambda x :str(x))
富台指數成分股['Code'] = 富台指數成分股['Code'].apply(lambda x :str(x))

加權指數成分股 = 加權指數成分股[['weight_%','Code']].merge(industry).fillna("其他")
富台指數成分股 = 富台指數成分股[['weight_%','Code']].merge(industry).fillna("其他")

加權減富台 = pd.DataFrame(加權指數成分股.groupby('TSE_Industry')['weight_%'].sum() - 富台指數成分股.groupby('TSE_Industry')['weight_%'].sum()).dropna().sort_values('weight_%')

加權前十 = pd.DataFrame(加權指數成分股.groupby('TSE_Industry')['weight_%'].sum()).sort_values('weight_%',ascending = False).head(10)
富台前十 = pd.DataFrame(富台指數成分股.groupby('TSE_Industry')['weight_%'].sum()).sort_values('weight_%',ascending = False).head(10)
富台台指差別 = (富台前十 - 加權前十).sort_values('weight_%',ascending = False)

ws = sh.worksheet_by_title('每日台指富台前十大產業權重')

ws.clear()

ws.update_value('A1:B1', "台指前十大產業權重")
ws.update_value('D1:E1', "富台前十大產業權重")
ws.update_value('G1:H1', "富台 - 台指")

ws.update_value('J1:N1', "更新時間 : %s " %(str(dt.datetime.today())[0:19]))

ws.set_dataframe(加權前十.apply(lambda x : np.around(x,4)*100).replace("nan",""), 'A2', copy_index=True, nan='')
ws.set_dataframe(富台前十.apply(lambda x : np.around(x,4)*100).replace("nan",""), 'D2', copy_index=True, nan='')
ws.set_dataframe(富台台指差別.apply(lambda x : np.around(x,4)*100).replace("nan",""), 'G2', copy_index=True, nan='')
# ws.set_dataframe(權指股前五筆開會.replace("nan",""), 'A2', copy_index=False, nan='')

print("除息監控")
### 除席監控 
DQ_stock_dde=PyWinDDE.DDEClient("DQII", "TWSE")
DQ_index_dde=PyWinDDE.DDEClient("DQII", "INDX")
DQ_future_dde=PyWinDDE.DDEClient("DQII", "FUSA")

path = '//192.168.60.81/Wellington/Wayne/除息預估/除息總整理/'+today+"/"
今日預估公司 = pd.read_csv(path+"全除息個股序列總表_"+today+".csv")
監督合約 = 台指合約_list[台指合約_list.index(TWN_contract_month(today))+1:台指合約_list.index(TWN_contract_month(today))+4]
今日預估公司 = 今日預估公司[(今日預估公司['除息日'].apply(lambda x : pd.to_datetime(x)) >= pd.to_datetime(today)) & 
                (今日預估公司['台指合約月份'].isin(監督合約)) &今日預估公司['台指影響點數']>0].reset_index(drop = True)
單純合約除息 = {}
累積合約除息 = {}

for 合約月份 in 監督合約 :
    單純合約除息["單純{}月合約".format(合約月份[6:7])] = np.around(今日預估公司[今日預估公司["台指合約月份"] == 合約月份]["台指影響點數"].sum(),4)
    
單純合約除息_df = pd.DataFrame(單純合約除息,index = ['除息點數']).T

for 合約月份 in 監督合約 :
    累積合約除息["累積{}月合約".format(合約月份[6:7])] = np.around(今日預估公司.loc[:今日預估公司[今日預估公司["台指合約月份"] == 合約月份].index[-1]]["台指影響點數"].sum(),4)
    
累積合約除息_df = pd.DataFrame(累積合約除息,index = ['除息點數']).T

期貨價格 = {}
期貨合約代碼 = ['WTXK3.129','WTXM3.129','WTXN3.129']
for ind , 合約月份 in enumerate(監督合約) :
    期貨價格["台指{}月期貨昨收".format(合約月份[6:7])] = float(DQ_future_dde.request(期貨合約代碼[ind]).decode()) # 5

期貨價格_df = pd.DataFrame(期貨價格,index = ['昨收']).T

期貨價差 = {}
for ind , 合約月份 in enumerate(監督合約[1:]) :
    期貨價差["{}月{}月價差".format(監督合約[0][6:7],合約月份[6:7])] = 期貨價格_df.昨收[0] - 期貨價格_df.昨收[ind+1]
    
期貨價差_df = pd.DataFrame(期貨價差,index = ['價差']).T

ws = sh.worksheet_by_title('除權息監控')

ws.clear()

ws.update_value('A1:E1', "近三個合約月 除息監控 更新時間 : {}".format(str(dt.datetime.today())[0:19]))

ws.set_dataframe(單純合約除息_df.replace("nan",""), 'A2', copy_index=True, nan='')
ws.set_dataframe(累積合約除息_df.replace("nan",""), 'A7', copy_index=True, nan='')
ws.set_dataframe(期貨價格_df.replace("nan",""), 'D2', copy_index=True, nan='')
ws.set_dataframe(期貨價差_df.replace("nan",""), 'D7', copy_index=True, nan='')


TX_Use:B06_TWTANU_TAI-20230712-C.XLS 
TWN_Use:FTCRTWRP as of Jul 13 20231.xlsx 
MTW_Use:tw_performance20230712.xls
除息監控


NameError: name 'today' is not defined

In [2]:
from datetime import datetime
current_time = datetime.now().today()
print(type(current_time.year))
current_time.year - 1

<class 'int'>


2022